# import pacakge and set up environment

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI 
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
import faiss
import pickle
import os
import requests
import json
from bs4 import BeautifulSoup
os.environ["OPENAI_API_BASE"] = "https://api.chatanywhere.tech/v1"
os.environ["OPENAI_API_KEY"] = "sk-X6qmvKSqTxaeFIrmq9v2KlAN6QJEKwxi6eGX5ads7wYERbx0"

# Loading data from Strikingly 

In [11]:
r = requests.get("https://support.strikingly.com/api/v2/help_center/en-us/articles.json")
articles_info = json.loads(r.text)['articles']
pages = []

def extract_text_from(html):
    soup = BeautifulSoup(html, features="html.parser")
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    return '\n'.join(line for line in lines if line)
for article in articles_info:
    print(article.keys())
    pages.append({'text': extract_text_from(article['body']), 'source': article['html_url']})
    

dict_keys(['id', 'url', 'html_url', 'author_id', 'comments_disabled', 'draft', 'promoted', 'position', 'vote_sum', 'vote_count', 'section_id', 'created_at', 'updated_at', 'name', 'title', 'source_locale', 'locale', 'outdated', 'outdated_locales', 'edited_at', 'user_segment_id', 'permission_group_id', 'content_tag_ids', 'label_names', 'body'])
dict_keys(['id', 'url', 'html_url', 'author_id', 'comments_disabled', 'draft', 'promoted', 'position', 'vote_sum', 'vote_count', 'section_id', 'created_at', 'updated_at', 'name', 'title', 'source_locale', 'locale', 'outdated', 'outdated_locales', 'edited_at', 'user_segment_id', 'permission_group_id', 'content_tag_ids', 'label_names', 'body'])
dict_keys(['id', 'url', 'html_url', 'author_id', 'comments_disabled', 'draft', 'promoted', 'position', 'vote_sum', 'vote_count', 'section_id', 'created_at', 'updated_at', 'name', 'title', 'source_locale', 'locale', 'outdated', 'outdated_locales', 'edited_at', 'user_segment_id', 'permission_group_id', 'content

# Process data

In [12]:


text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
docs = []
for page in pages:
    splits = text_splitter.split_text(page['text'])
    # 将每个文本片段转换为 Document 对象
    for split in splits:
        doc = Document(page_content=split, metadata={"source": page['source']})
        docs.append(doc)
    print(f"Split {page['source']} into {len(splits)} chunks")

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)


Split https://support.strikingly.com/hc/en-us/articles/33571866019995-Adding-Images-to-Your-Website into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/33195129962651-Strikingly-s-AI-Logo-Maker into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/32500484566811-Strikingly-AI-Site-Builder into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/30097621863835-Strikingly-Email-Creating-Filters-on-Webmail into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/25888063899931-How-do-I-delete-a-form into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/25585768352667-Can-I-upload-a-pdf-file-document-for-my-visitors-to-download into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/25511392983195-Can-my-website-carry-my-name into 1 chunks
Split https://support.strikingly.com/hc/en-us/articles/25511191343899-How-can-I-forward-redirect-my-domain-from-another-registrar-to-my-Strikingly-domain into 1 chunks
Sp

# Method 1

In [13]:

qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0), vectorstore.as_retriever(), return_source_documents=True)
use_history = False
chat_history = []


query = "How to add images to your website"
result = qa({"question": query, "chat_history": chat_history if use_history else []})
chat_history = [(query, result["answer"])]

print('Answer:'+result["answer"]+"\n")
print('Sources:')
for doc in result["source_documents"]:
    print(doc.metadata["source"])

Answer:To add images to your website on Strikingly, you can follow these steps:

1. Adding Images in Sections:
- In the Site Editor, click on "Add New Section" on the left side of the screen.
- Under the “Image & Video” category, choose the section you want to add images to (Big Media, Banner Slider, Grid, etc.).
- Upload your image(s) and adjust settings as needed.

2. Adding Images to Other Sections:
- You can add images to sections like Gallery, Team, Testimonials, Feature List, Portfolio, Store, and Blog.
- For example, in the Gallery Section, click "Manage Gallery" > "Import Image/Video" to add images.

3. Adding Images to Backgrounds:
- In the Site Editor, click "Background" in the section you want to edit.
- Click on “Image” and upload the image you want to use as the background.
- Adjust settings like positioning and scaling for a polished look.

Remember to ensure your images are high-quality and appropriately sized for the best results.

Sources:
https://support.strikingly.co

# Method 2

In [14]:

llm = ChatOpenAI(temperature=0)

question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_with_sources_chain(llm)

chain = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
)

chat_history = []
query = "How to add images to your website"
result = chain({"question": query, "chat_history": chat_history if use_history else []})
chat_history.append((query, result["answer"]))
answer = result['answer'].split("SOURCES:")[0]
sources = result['answer'].split("SOURCES:")[1]


print(f"Answer: {answer}\n")
print(f"Sources: {sources}")

Answer: To add images to your website, you can follow the step-by-step guide provided by Strikingly. Additionally, you can use the AI Site Builder feature to generate websites quickly. 


Sources:  
- https://support.strikingly.com/hc/en-us/articles/33571866019995-Adding-Images-to-Your-Website
- https://support.strikingly.com/hc/en-us/articles/32500484566811-Strikingly-AI-Site-Builder
